In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

from typing import Union

pd.options.mode.chained_assignment = None

In [ ]:
df= pd.read_csv('athlete_events.csv')
region_df = pd.read_csv('noc_regions.csv')

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df=df[df['Season']=='Summer']

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
region_df.tail()

In [ ]:
df.merge(region_df,on = 'NOC', how='left')

In [ ]:
df.tail()

In [ ]:
df['region'].unique()

In [ ]:
df.isnull().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.duplicated().sum()

In [ ]:
df['Medal'].value_counts()

In [ ]:
pd.get_dummies(df['Medal'])

In [ ]:
df.shape

In [ ]:
df.tail()

In [ ]:
df.groupby('NOC').sum()[['Gold','Silver','Bronze']]

In [ ]:
# check duplicate rows
total_duplicate_rows = df.duplicated().sum()
print(f"Total duplicated rows {total_duplicate_rows}")

# drop those duplicated rows
df.drop_duplicates(inplace=True)

In [ ]:
# now "Medal" column
df["Medal"].value_counts()

In [ ]:
# create OneHotEncodded column for 3 medal types
print(f"Before OHE, the shape is {df.shape}")
df = pd.concat([df, pd.get_dummies(df["Medal"])], axis=1)
print(f"After OHE, the shape is {df.shape}")

In [ ]:
df.groupby("NOC").sum()[["Gold", "Silver", "Bronze"]].sort_values("Gold", ascending=False).reset_index()

## Is there anything wrong with this data?

In [ ]:
# If you get confused, then see this filter data
df[(df["NOC"] == "IND") & (df["Medal"] == "Gold")].head(5)

In [ ]:
# now time to solve this problem
medal_tally = df.drop_duplicates(subset=["Team", "NOC", "Games", "Year", "City", "Sport", "Event", "Medal"])
medal_tally.shape

In [ ]:
medal_tally = medal_tally.groupby("NOC").sum()[["Gold", "Silver", "Bronze"]].sort_values("Gold", ascending=False).reset_index()
medal_tally.head(5)
# till now there is a problem but the data is close to real one.

In [ ]:
medal_tally[medal_tally["NOC"] == "IND"]

In [ ]:
medal_tally = df.drop_duplicates(subset=["Team", "NOC", "Games", "Year", "City", "Sport", "Event", "Medal"])
medal_tally = medal_tally.groupby("region").sum()[["Gold", "Silver", "Bronze"]].sort_values("Gold", ascending=False).reset_index()
medal_tally["total"] = medal_tally["Gold"] + medal_tally["Silver"] + medal_tally["Bronze"]
medal_tally.head()

In [ ]:
years = df["Year"].unique().tolist()
years.sort()
years.insert(0, "Overall")
years


In [ ]:
country = np.unique(df["region"].dropna().values).tolist()
country.sort()
country.insert(0, "Overall")

In [ ]:
def fetch_medal_tally(df: pd.DataFrame, year: Union[int, str], country: Union[int, str]) -> pd.DataFrame:
    flag = 0
    medal_df = df.drop_duplicates(subset=["Team", "NOC", "Games", "Year", "City", "Sport", "Event", "Medal"])
    if year == "Overall" and country == "Overall":
        temp_df = medal_df
    elif year == "Overall" and country != "Overall":
        flag = 1
        temp_df = medal_df[medal_df["region"] == country]
    elif year != "Overall" and country == "Overall":
        temp_df = medal_df[medal_df["Year"] == year]
    else:
        temp_df = medal_df[(medal_df["Year"] == year) & (medal_df["region"] == country)]
    
    if flag == 1:
        x = temp_df.groupby("Year").sum()[["Gold", "Silver", "Bronze"]].sort_values("Year", ascending=True).reset_index()
        x["total"] = x["Gold"] + x["Silver"] + x["Bronze"]
    else:
        x = temp_df.groupby("region").sum()[["Gold", "Silver", "Bronze"]].sort_values("Gold", ascending=False).reset_index()
        x["total"] = x["Gold"] + x["Silver"] + x["Bronze"]
    
    return x

In [ ]:
fetch_medal_tally(df, year = 1900, country="India")

In [ ]:
df["Year"].unique()

In [ ]:
df["City"].unique()

In [ ]:
df["Sport"].unique()

In [ ]:
df["Event"].unique().shape

In [ ]:
df["Name"].unique().shape

In [ ]:
df["region"].unique().shape

In [ ]:
nations_over_time = df.drop_duplicates(["Year", "region"])["Year"].value_counts().reset_index().sort_values("index")
nations_over_time.rename(columns = {"index": "Edition", "Year": "No. of Countries"})

In [ ]:
fig = px.line(nations_over_time, x = "Edition", y = "No. of Countries")
fig.show()

In [ ]:
x = df.drop_duplicates(["Year", "Sport", "Event"])

plt.figure(figsize = (16, 16))
sns.heatmap(x.pivot_table(index="Sport", columns="Year", values="Event", aggfunc="count").fillna(0).astype("int"), annot=True)
plt.show()

In [ ]:
def most_successful(df: pd.DataFrame, sport: str) -> pd.DataFrame:
    temp_df = df.dropna(subset=["Medal"])
    
    if sport != "Overall":
        temp_df = temp_df[temp_df["Sport"] == sport]
    temp_df = temp_df["Name"].value_counts().reset_index().head(15).merge(df, left_on="index", right_on="Name", how="left")
    x = temp_df[["index", "Name_x", "Sport", "region"]].drop_duplicates("index")
    x.rename(columns = {"index": "Name", "Name_x": "Medals"}, inplace = True)
    return x

In [ ]:
most_successful(df, "Fencing")

In [ ]:
temp_df = df.dropna(subset=["Medal"])
temp_df.drop_duplicates(subset=["Team", "NOC", "Games", "Year", "City", "Sport", "Event", "Medal"], inplace=True)
new_df   = temp_df[temp_df["region"] == "India"]
final_df = new_df.groupby("Year").count()["Medal"].reset_index()

fig = px.line(final_df, x="Year", y="Medal")
fig.show()

In [ ]:
temp_df = df.dropna(subset=["Medal"])
temp_df.drop_duplicates(subset=["Team", "NOC", "Games", "Year", "City", "Sport", "Event", "Medal"], inplace=True)
new_df   = temp_df[temp_df["region"] == "UK"]

plt.figure(figsize=(16, 16))
pivot_table = new_df.pivot_table(index="Sport", columns="Year", values="Medal", aggfunc="count").fillna(0)
print(f"Type of pivot table: {type(pivot_table)}")
sns.heatmap(pivot_table, annot=True)
plt.show()

In [ ]:
def most_successful(df: pd.DataFrame, country: str) -> pd.DataFrame:
    temp_df = df.dropna(subset=["Medal"])
    
    
    temp_df = temp_df[temp_df["region"] == country]
    
    temp_df = temp_df["Name"].value_counts().reset_index().head(15).merge(df, left_on="index", right_on="Name", how="left")
    x = temp_df[["index", "Name_x", "Sport"]].drop_duplicates("index")
    x.rename(columns = {"index": "Name", "Name_x": "Medals"}, inplace = True)
    return x

In [ ]:
most_successful(df, "Jamaica")
Name	Medals	Sport
0	Merlene Joyce Ottey-Page	9	Athletics
19	Veronica Angella Campbell-Brown	8	Athletics
30	Usain St. Leo Bolt	8	Athletics
40	Shelly-Ann Fraser-Pryce	6	Athletics
47	Donald O'Reilly "Don" Quarrie	4	Athletics
56	Herbert Henry "Herb" McKenley	4	Athletics
62	Arthur Stanley Wint	4	Athletics
68	Novlene Hilaire Williams-Mills	4	Athletics
75	Yohan Blake	4	Athletics
81	Sherone Anmarica Simpson	3	Athletics
88	Juliet Jean Cuthbert	3	Athletics
98	Kerron Stewart	3	Athletics
103	Elaine Thompson	3	Athletics
106	Shericka Nicola Williams	3	Athletics
109	Beverly McDonald (-Stewart)	3	Athletics

In [ ]:
athlete_df = df.drop_duplicates(subset=["Name", "region"])
x1 = athlete_df["Age"].dropna()
x2 = athlete_df[athlete_df["Medal"] == "Gold"]["Age"].dropna()
x3 = athlete_df[athlete_df["Medal"] == "Silver"]["Age"].dropna()
x4 = athlete_df[athlete_df["Medal"] == "Bronze"]["Age"].dropna()

fig = ff.create_distplot([x1, x2, x3, x4], ["Overall Age", "Gold Medalist", "Silver Medalist", "Brownze Medalist"], show_hist=False, show_rug=False)
fig.show()

In [ ]:
famous_sports = ['Basketball', 'Judo', 'Football', 'Tug-Of-War', 'Athletics',
                     'Swimming', 'Badminton', 'Sailing', 'Gymnastics',
                     'Art Competitions', 'Handball', 'Weightlifting', 'Wrestling',
                     'Water Polo', 'Hockey', 'Rowing', 'Fencing',
                     'Shooting', 'Boxing', 'Taekwondo', 'Cycling', 'Diving', 'Canoeing',
                     'Tennis', 'Golf', 'Softball', 'Archery',
                     'Volleyball', 'Synchronized Swimming', 'Table Tennis', 'Baseball',
                     'Rhythmic Gymnastics', 'Rugby Sevens',
                     'Beach Volleyball', 'Triathlon', 'Rugby', 'Polo', 'Ice Hockey']

x    = []
name = []

for sport in famous_sports:
    temp_df = athlete_df[athlete_df["Sport"] == sport]
    x.append(temp_df[temp_df["Medal"] == "Gold"]["Age"].dropna())
    name.append(sport)
    
fig = ff.create_distplot(x, name, show_hist=False, show_rug=False)
fig.show()

In [ ]:
athlete_df["Medal"].fillna("No Medal", inplace=True)

plt.figure(figsize = (10, 10))
temp_df = athlete_df[athlete_df["Sport"] == "Athletics"]
sns.scatterplot(x=temp_df["Weight"], y=temp_df["Height"], hue=temp_df["Medal"], style=temp_df["Sex"], s=100)
plt.show()

In [ ]:
men   = athlete_df[athlete_df["Sex"] == "M"].groupby("Year").count()["Name"].reset_index()
women = athlete_df[athlete_df["Sex"] == "F"].groupby("Year").count()["Name"].reset_index()

final = men.merge(women, on="Year", how="left").fillna(0)
final.rename(columns={"Name_x":"Male", "Name_y":"Female"}, inplace=True)

fig = px.line(final, x="Year", y=["Male", "Female"])
fig.show()